# Experiments

In [ ]:
import os
import pandas as pd
import time
import torch
from configs.models import SCOOP_cfg, NeuralPrior_cfg
from data.gpu_utils import get_free_gpu_indices
from pipeline.run_utils import run_experiment

# load cfg list from csv later? or just permute variables
cfg_list = [#{**SCOOP_cfg, 'affiliation' : 'baseline', 'smooth_weight' : 1},
            {**NeuralPrior_cfg, 'affiliation' : 'ours', 'sm_normals_K' : 5, 'K' : 4, 'smooth_weight' : 1., 'forward_weight' : 1, 'free_weight' : 1, 'pc2_smooth' : 1, 'VA' : 1},
            ]

import subprocess
DETACH = 0
# tohle cele do subprocessu ve funkci potom #



for cfg in cfg_list:

    while len(get_free_gpu_indices()) == 0:
        print('waiting for gpu')
        time.sleep(20)

    # for dataset in ['kitti_o', 'kitti_t', 'argoverse', 'nuscenes', 'waymo']:
    for dataset in ['argoverse']:

        cfg['dataset'] = dataset
        cfg['iters'] = 100
        # cfg['lr'] = 0.008
        cfg['dev'] = 1
        # cfg['vis'] = 1
        # print(dataset)
        run_experiment(cfg, DETACH=DETACH)

    # time.sleep(5)

    # break

# notes
- SCOOP wont generalize to other real world datasets (Cross-dataset evaluation) - performance on argoverse is bad, retraining?
    - "It extracts discriminative features, which transfer well across the FT3Do and KITTIo datasets, and enables to compute the correspondence-based flow between the point clouds"
    - We dont have advantage in that, but this kinda justifies the results on argoverse without fine-tunning?
    - KittiSF is bijection, that is why it works very good in SCOOP
- https://arxiv.org/pdf/2305.02528.pdf cite baseline
- Pouceni z reimplementace
    - Asi to fakt chce delat krok po kroku uz odzacatku? Tim mysleno testovat vsechno krok po kroku kdyz se prevezmou funkce
    - Izolovat (rozumet tem krokum, jinak je proste nedelat, opravdu nedelat)
    - Skvely je mit ten framework udelany podle toho, co opravdu clovek delat
    - Nejlepsi je mit vsechno v jednom editoru viz. poustet pre python subprocessy, jinak prepinani zpomaluje a nici to framework workflow


In [27]:
# Compare experiments, # Udelat cross experiment vizualizaci
import glob
import pandas as pd
import os
import matplotlib.pyplot as plt
from data.PATHS import TMP_VIS_PATH

exp_name = "grid_search"
exp_dir = f'{os.path.expanduser("~")}/experiments/{exp_name}'
runs = sorted(glob.glob(f'{exp_dir}/*'))

# plot?

metric_all = []
fig, axes = plt.subplots(1, 2)#, figsize=(10, 5))

# plt.close()
# plt.clf()


for run in runs:

    if os.path.exists(f'{run}/metric.csv') == False:
        continue

    metric = pd.read_csv(f'{run}/metric.csv', index_col=0, header=None).transpose()
    metric = [v.to_dict() for k,v in metric.iterrows()][0]

    args = pd.read_csv(f'{run}/args.csv', header=None, index_col=0).transpose()
    args = [v.to_dict() for k, v in args.iterrows()][0]

    for k in ['VA', 'free_weight', 'pc2_smooth', 'smooth_weight', 'forward_weight', 'sm_normals_K']:
        if k in args.keys():
            args[k] = float(args[k])

    if args['dataset'] == 'kitti_o': continue

    # Skip tryouts
    if args['dev'] == 1:
        continue

    # aff = args['affiliation'].values[0]
    epe = metric['EPE3D']
    avg_solver_time = metric['avg_solver_time']
    # decide about aff
    if args['model'] == 'NeuralPrior':

        if args['VA'] == 0 and args['free_weight'] == 0 and args['forward_weight'] == 0 and args['pc2_smooth'] == 0 and args['smooth_weight'] == 0:
            aff = 'baseline'
        else:
            aff = 'ours'

    if args['model'] == 'SCOOP':
        if args['sm_normals_K'] == 0 and args['VA'] == 0 and args['free_weight'] == 0 and args['forward_weight'] == 0 and args['pc2_smooth'] == 0:
            aff = 'baseline'
        else:
            aff = 'ours'

    # if aff == 'baseline':
    #     continue

    del args['affiliation']
    args['aff'] = aff
    model_marker = '*' if args['model'] == 'NeuralPrior' else 'o'
    color_marker = 'g' if aff == 'ours' else 'b'


    axes[0].plot(avg_solver_time, epe, color_marker + model_marker, markersize=4, label=args['model'])


    full_dict = {**args, **metric}
    name_weight_list = ['free_weight', 'smooth_weight', 'forward_weight', 'pc2_smooth', 'sm_normals_K', 'VA']
    name_metrics = ['EPE3D', 'acc3d_strict', 'acc3d_relax', 'angle_error', 'outlier']
    interest_dict = {k:v for k,v in full_dict.items() if k in ['aff', 'avg_solver_time', 'model', 'EPE3D'] + name_weight_list + name_metrics}

    # metric_all.append(full_dict)
    metric_all.append(interest_dict)

axes[0].legend(['NeuralPrior', 'SCOOP'])

df = pd.DataFrame(metric_all)



axes[0].set_xlabel('Average Solver Time Per Frame [s]')
axes[0].set_ylabel('EPE [m]')

axes[0].set_title('Performance on KITTI-SF')
axes[0].legend(['baseline', 'ours'])
axes[0].grid(True)


#create pandas DataFrame

#create table
# table = plt.table(cellText=df.values, colLabels=df.columns, loc='center')
plt.savefig(f'{TMP_VIS_PATH}/kittisf.png')
plt.close()

df
# standard deviation?



,model,smooth_weight,forward_weight,free_weight,VA,sm_normals_K,pc2_smooth,aff,EPE3D,acc3d_strict,acc3d_relax,angle_error,outlier,avg_solver_time
0,NeuralPrior,1.0,1.0,0.0,0.0,8.0,1.0,ours,0.038202,0.895604,0.968941,0.127274,0.150882,3.377815
1,NeuralPrior,0.0,0.0,0.0,0.0,0.0,0.0,baseline,0.035799,0.915465,0.959723,0.122890,0.130793,2.608330
2,NeuralPrior,1.0,0.0,0.0,0.0,0.0,0.0,ours,0.030873,0.936751,0.978280,0.111395,0.121046,2.473765
3,NeuralPrior,0.0,0.0,1.0,0.0,0.0,0.0,ours,0.032693,0.919955,0.974072,0.116542,0.108876,2.660900
4,NeuralPrior,0.0,0.0,0.0,1.0,0.0,0.0,ours,0.035892,0.928049,0.975277,0.120856,0.142548,2.406570
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
234,SCOOP,10.0,10.0,1.0,0.0,12.0,0.0,ours,0.024888,0.976342,0.986366,0.113298,0.119376,4.175589
235,SCOOP,10.0,10.0,1.0,1.0,12.0,0.0,ours,0.024854,0.976516,0.986793,0.113396,0.119132,4.261738
236,SCOOP,10.0,10.0,0.0,0.0,12.0,1.0,ours,0.024473,0.977752,0.985462,0.112696,0.116760,3.965773
237,SCOOP,10.0,10.0,0.0,1.0,12.0,1.0,ours,0.024893,0.977690,0.985251,0.112888,0.116859,4.035503


In [28]:
# for each of the datasets
# when specific loss is used, plot everything
# maybe add even rest of the metrics
df.loc[df['model'] == 'SCOOP']


,model,smooth_weight,forward_weight,free_weight,VA,sm_normals_K,pc2_smooth,aff,EPE3D,acc3d_strict,acc3d_relax,angle_error,outlier,avg_solver_time
160,SCOOP,10.0,0.0,0.0,0.0,0.0,0.0,baseline,0.026485,0.973587,0.983764,0.114288,0.117709,2.959095
161,SCOOP,10.0,0.0,0.0,1.0,0.0,0.0,ours,0.026542,0.972849,0.983769,0.114296,0.117369,3.017039
162,SCOOP,10.0,0.0,1.0,0.0,0.0,0.0,ours,0.025996,0.975825,0.984730,0.114393,0.118123,2.970058
163,SCOOP,10.0,0.0,1.0,1.0,0.0,0.0,ours,0.026718,0.973470,0.982736,0.114534,0.119662,3.156393
164,SCOOP,10.0,1.0,0.0,0.0,0.0,0.0,ours,0.026127,0.973418,0.984388,0.113924,0.117184,4.263794
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
234,SCOOP,10.0,10.0,1.0,0.0,12.0,0.0,ours,0.024888,0.976342,0.986366,0.113298,0.119376,4.175589
235,SCOOP,10.0,10.0,1.0,1.0,12.0,0.0,ours,0.024854,0.976516,0.986793,0.113396,0.119132,4.261738
236,SCOOP,10.0,10.0,0.0,0.0,12.0,1.0,ours,0.024473,0.977752,0.985462,0.112696,0.116760,3.965773
237,SCOOP,10.0,10.0,0.0,1.0,12.0,1.0,ours,0.024893,0.977690,0.985251,0.112888,0.116859,4.035503


In [5]:
import itertools
import pandas as pd
import os
from configs.models import SCOOP_cfg, NeuralPrior_cfg
### Make configs

permute_cfg = {'model': ['NeuralPrior', 'SCOOP'],
               'store_inference' : [0],
               'sm_normals_K': [0, 5, 8, 12],
               'forward_weight':  [0, 1, 10],
               'pc2_smooth': [0, 1],
               'K': [4, 32],
               'smooth_weight' : [0, 1],
               'free_weight': [0, 1],
               'VA': [0, 1],
               'lr': [0.2, 0.008],
               'dataset': ['kitti_t', 'kitti_o'],
               'max_radius' : [2],
               'early_patience' : [50],
               }

# 'dataset': ['kitti_t', 'kitti_o', 'argoverse', 'nuscenes', 'waymo'],   # wait for lidar on tommorow
# permute all possible combinations of variables above

index = permute_cfg.keys()
combinations = list(itertools.product(*permute_cfg.values()))

df = pd.DataFrame(combinations, columns=index)

valid_cfg_list = []


# leave out some configs
runs = 3
for run in range(runs):

    for i in range(len(combinations)):
        default_cfg = {}
        c = df.iloc[i]

        ##### KittiSF
        if c['dataset'].startswith('kitti'):

            if c['K'] != 32: continue

        ##### Lidar
        if not c['dataset'].startswith('kitti'):

            if c['K'] > 8: continue
            if c['sm_normals_K'] > 5: continue

        if c['dataset'] in ['argoverse', 'nuscenes', 'waymo'] and c['sm_normals_K'] > 5: continue

        ##### Models
        if c['model'] == "SCOOP":
            default_cfg = SCOOP_cfg
            if c['K'] != 32: continue
            if c['lr'] != 0.2: continue
            if c['dataset'] in ['argoverse', 'nuscenes', 'waymo']: continue
            if c['smooth_weight'] != 1: continue

        if c['model'] == 'NeuralPrior':
            default_cfg = NeuralPrior_cfg
            if c['dataset'] in ['kitti_o']: continue
            if c['lr'] == 0.2: continue
            # if c['lr'] != 0.001 and c['dataset'] == 'kitti_t': continue
            # if c['lr'] != 0.003 and c['dataset'] in ['argoverse', 'waymo', 'nuscenes']: continue


        ##### General
        if c['forward_weight'] == 0 and c['pc2_smooth'] == 1: continue
        if c['K'] == 0 and (c['forward_weight'] > 0 or c['VA'] > 0): continue


        final_config = {**default_cfg, **c.to_dict()}
        final_config['exp_name'] = final_config['model']
        final_config['dev'] = 0

        # Exceptions
        if c['model'] == "SCOOP":
            final_config['smooth_weight'] = default_cfg['smooth_weight']
            final_config['iters'] = 150

        if c['model'] == "NeuralPrior":
            final_config['iters'] = 250

        # Just to make them more and compute multiple times. Otherwise it is not in argparser
        valid_cfg_list.append(final_config)

print(len(valid_cfg_list))
nbr_gpu = 5
experiment_time = 10 / 60 # hours
gpu_time = len(valid_cfg_list) * experiment_time

print(f"Runs: {len(valid_cfg_list)} ---> GPU time: {gpu_time:.1f} hours ---> {gpu_time / nbr_gpu / 24:.1f} days")

final_df = pd.DataFrame(valid_cfg_list, columns=index)
# save to different formats
final_df.to_csv(f'{os.path.expanduser("~")}/pcflow/configs/experiments/grid_search.csv', index=False)

960
Runs: 960 ---> GPU time: 160.0 hours ---> 1.3 days


In [ ]:

# Top down view example of NN connections
KNN = nn_ind[0]
start_arrows = pc2[0][KNN].reshape(-1,3)

# NN connections
residuals_KNN = (pc2[0].unsqueeze(1) - pc2[0][KNN]).reshape(-1,3)
residuals_VA_KNN = (pc2[0].unsqueeze(1) - pc2[0][VA_KNN]).reshape(-1,3)

import matplotlib.pyplot as plt
from data.PATHS import TMP_VIS_PATH


fig, ax = plt.subplots(1,2, figsize=(10,5))
ax[0].plot(start_arrows[:,0].cpu(), start_arrows[:,1].cpu(), 'bo', alpha=0.15, markersize=0.1)
ax[1].plot(start_arrows[:,0].cpu(), start_arrows[:,1].cpu(), 'bo', alpha=0.15, markersize=0.1)

ax[0].quiver(start_arrows[:,0].cpu(), start_arrows[:,1].cpu(), residuals_KNN[:,0].cpu(), residuals_KNN[:,1].cpu(), color='r', scale_units='xy')
ax[1].quiver(start_arrows[:,0].cpu(), start_arrows[:,1].cpu(), residuals_VA_KNN[:,0].cpu(), residuals_VA_KNN[:,1].cpu(), color='r', scale_units='xy')

ax[0].axis('equal')
ax[1].axis('equal')

plt.savefig(f'{TMP_VIS_PATH}/{dataset_type}_range.png')


['/home.dokt/vacekpa2/experiments/dev/2023-07-18-20-13-06-618/', '/home.dokt/vacekpa2/experiments/dev/2023-07-18-20-13-06-623/', '/home.dokt/vacekpa2/experiments/dev/2023-07-18-20-13-06-644/', '/home.dokt/vacekpa2/experiments/dev/2023-07-18-20-13-06-645/', '/home.dokt/vacekpa2/experiments/dev/2023-07-18-20-13-06-681/', '/home.dokt/vacekpa2/experiments/dev/2023-07-18-20-13-06-788/', '/home.dokt/vacekpa2/experiments/dev/2023-07-18-20-13-06-852/', '/home.dokt/vacekpa2/experiments/dev/2023-07-18-20-30-16-033/', '/home.dokt/vacekpa2/experiments/dev/2023-07-18-20-30-16-765/', '/home.dokt/vacekpa2/experiments/dev/2023-07-18-20-30-43-153/', '/home.dokt/vacekpa2/experiments/dev/2023-07-18-20-39-46-630/', '/home.dokt/vacekpa2/experiments/dev/2023-07-18-20-40-20-900/', '/home.dokt/vacekpa2/experiments/dev/2023-07-18-20-41-42-641/', '/home.dokt/vacekpa2/experiments/dev/2023-07-18-20-43-07-482/', '/home.dokt/vacekpa2/experiments/dev/2023-07-18-20-50-17-780/', '/home.dokt/vacekpa2/experiments/dev/20